In [50]:
import pandas as pd
from footix.models.bayesian import Bayesian
import footix.implied_odds as odds
import numpy as np
from footix.data_io.data_scrapper import ScrapFootballData
from footix.strategy.strategies import realKelly
from footix.strategy.bets import OddsInput, Bet
from typing import Literal
from footix.strategy.select_bets import simple_select_bets, select_matches_posterior
import json

In [51]:
dataset = ScrapFootballData(competition="ligue2", season="2425", path ="./data", force_reload=True).get_data()

In [52]:
dataset.head(-5)

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,BFECAHH,BFECAHA
0,F2,16/08/2024,19:00,Ajaccio,Rodez,1,0,H,0,0,...,1.93,1.93,1.97,1.92,2.02,1.96,1.96,1.89,2.00,1.99
1,F2,16/08/2024,19:00,Amiens,Red Star,3,0,H,1,0,...,2.13,1.75,2.17,1.76,2.17,1.84,2.07,1.76,2.19,1.83
2,F2,16/08/2024,19:00,Clermont,Pau FC,2,2,D,1,2,...,1.83,2.03,1.86,2.03,1.89,2.10,1.80,2.02,1.89,2.10
3,F2,16/08/2024,19:00,Dunkerque,Annecy,0,2,A,0,0,...,1.78,2.10,1.75,2.16,1.83,2.22,1.72,2.13,1.82,2.20
4,F2,16/08/2024,19:00,Grenoble,Laval,2,1,H,0,0,...,2.00,1.85,2.01,1.88,2.02,1.92,1.97,1.84,2.05,1.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,F2,14/04/2025,19:45,Bastia,Laval,5,2,H,2,1,...,1.83,2.03,1.88,2.01,1.88,2.07,1.83,1.97,1.87,2.10
270,F2,18/04/2025,19:00,Ajaccio,Pau FC,1,1,D,0,1,...,2.10,1.78,2.16,1.75,2.16,1.79,2.10,1.74,2.22,1.79
271,F2,18/04/2025,19:00,Amiens,Guingamp,3,2,H,2,1,...,1.85,2.00,1.86,2.03,1.89,2.09,1.81,2.00,1.89,2.09
272,F2,18/04/2025,19:00,Caen,Martigues,0,3,A,0,2,...,1.93,1.93,1.95,1.93,1.96,1.98,1.91,1.90,1.92,2.03


In [53]:
dataset["HomeTeam"].unique()

array(['Ajaccio', 'Amiens', 'Clermont', 'Dunkerque', 'Grenoble',
       'Guingamp', 'Caen', 'Martigues', 'Metz', 'Annecy', 'Bastia',
       'Laval', 'Paris FC', 'Pau FC', 'Rodez', 'Troyes', 'Lorient',
       'Red Star'], dtype=object)

In [54]:
model = Bayesian(n_teams=18, n_goals=15)

In [55]:
model.fit(X_train=dataset)

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (6 chains in 6 jobs)
NUTS: [home, intercept, tau_att, raw_atts, tau_def, raw_defs]


Output()

Sampling 6 chains for 500 tune and 2_000 draw iterations (3_000 + 12_000 draws total) took 9 seconds.
There were 1 divergences after tuning. Increase `target_accept` or reparameterize.


In [58]:
mapping_dict = {
    'AC Ajaccio': 'Ajaccio',
    'Amiens': 'Amiens',
    'Clermont': 'Clermont',
    'Dunkerque': 'Dunkerque',
    'Grenoble': 'Grenoble',
    'Guingamp': 'Guingamp',
    'Caen': 'Caen',
    'FC Martigues': 'Martigues',
    'Metz': 'Metz',
    'Annecy FC': 'Annecy',
    'Bastia': 'Bastia',
    'Laval': 'Laval',
    'Paris FC': 'Paris FC',
    'Pau': 'Pau FC',
    'Rodez': 'Rodez',
    'Troyes': 'Troyes',
    'Lorient': 'Lorient',
    'Red Star': 'Red Star'
}


In [60]:
fixture = pd.read_csv("ligue2_24-04-2025.csv")
convert = True
fixture = fixture.iloc[:9]
# Assuming fixture and odds are already defined DataFrames
proba_match = []
list_odds = []
list_samples = {}
for idx, match in fixture.iterrows():
    print("#" * 50)
    print(f"{match['home_team']:<30} {match['away_team']:<30}")
    # Display probabilities from Poisson prediction
    home_team = mapping_dict.get(match["home_team"], match["home_team"])
    away_team = mapping_dict.get(match["away_team"], match["away_team"])

    probas = model.predict(home_team=home_team, away_team=away_team).return_probas()
    lambda_h, lambda_a = model.get_samples(home_team=home_team, away_team=away_team)
    home_prob = probas[0]
    draw_prob = probas[1]
    away_prob = probas[2]
    print(f"  Probabilities: Home: {home_prob:.2f}, Draw: {draw_prob:.2f}, Away: {away_prob:.2f}")
    # Display odds from the function
    odds_list = [match["H"], match["D"], match["A"]]
    proba_match.append(probas)
    list_odds.append(OddsInput(home_team=match["home_team"], away_team=match["away_team"], odds=odds_list))
    list_samples[list_odds[-1].match_id] = (lambda_h, lambda_a)

    odds_result = odds.shin(odds=odds_list)[0]
    print(f"  Odds: Home: {odds_result[0]:.2f}, Draw: {odds_result[1]:.2f}, Away: {odds_result[2]:.2f}")
    print("#" * 50)
proba_match = np.asarray(proba_match)

##################################################
Guingamp                       FC Martigues                  
  Probabilities: Home: 0.60, Draw: 0.21, Away: 0.19
  Odds: Home: 0.71, Draw: 0.18, Away: 0.12
##################################################
##################################################
Bastia                         Grenoble                      
  Probabilities: Home: 0.51, Draw: 0.25, Away: 0.23
  Odds: Home: 0.59, Draw: 0.24, Away: 0.17
##################################################
##################################################
Troyes                         Dunkerque                     
  Probabilities: Home: 0.35, Draw: 0.30, Away: 0.35
  Odds: Home: 0.40, Draw: 0.29, Away: 0.31
##################################################
##################################################
Laval                          Amiens                        
  Probabilities: Home: 0.47, Draw: 0.27, Away: 0.27
  Odds: Home: 0.51, Draw: 0.27, Away: 0.22
###############

In [61]:
list_bet = simple_select_bets(odds_input=list_odds, probas=proba_match, single_bet_per_game=True, edge_floor=0.15)

In [62]:
list_bet[:9]

[Bet(match_id='Guingamp - FC Martigues', market='A', odds=7.2, edge_mean=0.3862680466527022, prob_mean=0.1925372287017642, edge_std=None, prob_edge_pos=None, stake=None),
 Bet(match_id='Bastia - Grenoble', market='A', odds=5.2, edge_mean=0.22146989940231165, prob_mean=0.23489805757736762, edge_std=None, prob_edge_pos=None, stake=None),
 Bet(match_id='Red Star - Clermont', market='A', odds=3.65, edge_mean=0.25624883791361475, prob_mean=0.34417776381194926, edge_std=None, prob_edge_pos=None, stake=None),
 Bet(match_id='Rodez - Paris FC', market='H', odds=2.95, edge_mean=0.3373342143863767, prob_mean=0.4533336319953819, edge_std=None, prob_edge_pos=None, stake=None),
 Bet(match_id='Pau - Metz', market='H', odds=4.1, edge_mean=0.225257218658231, prob_mean=0.2988432240629832, edge_std=None, prob_edge_pos=None, stake=None)]

In [63]:
enhanced_list_bet = select_matches_posterior(odds_input=list_odds, lambda_samples = list_samples, edge_floor=0.1, prob_edge_threshold=0.7, single_bet_per_game=True)

In [64]:
enhanced_list_bet

[Bet(match_id='Guingamp - FC Martigues', market='A', odds=7.2, edge_mean=0.4147971590267324, prob_mean=0.1964996054203795, edge_std=0.447112360145378, prob_edge_pos=0.816, stake=None),
 Bet(match_id='Rodez - Paris FC', market='H', odds=2.95, edge_mean=0.3475090933698626, prob_mean=0.45678274351520765, edge_std=0.2852163463582172, prob_edge_pos=0.8885, stake=None),
 Bet(match_id='Red Star - Clermont', market='A', odds=3.65, edge_mean=0.2561424828445391, prob_mean=0.34414862543686, edge_std=0.2721221382494443, prob_edge_pos=0.8226666666666667, stake=None),
 Bet(match_id='Pau - Metz', market='H', odds=4.1, edge_mean=0.25548584409357866, prob_mean=0.3062160595350192, edge_std=0.3475483505361055, prob_edge_pos=0.75875, stake=None),
 Bet(match_id='Bastia - Grenoble', market='A', odds=5.2, edge_mean=0.23434221014949275, prob_mean=0.23737350195182555, edge_std=0.32937590555444957, prob_edge_pos=0.7481666666666666, stake=None)]

In [65]:
selected_bet = realKelly(enhanced_list_bet[:9], bankroll=20, max_multiple=1, early_stopping=False)

  0%|          | 0/1000 [00:00<?, ?it/s]


2025-04-24 22:57:27- Optimization finished. Runtime --- 1.906 seconds ---

Objective: -3.06299
Certainty Equivalent: 21.391

[Guingamp - FC Martigues | A] odds=7.20, edge=0.415, p=0.196, stake=1.00
[Rodez - Paris FC | H] odds=2.95, edge=0.348, p=0.457, stake=3.00
[Red Star - Clermont | A] odds=3.65, edge=0.256, p=0.344, stake=2.00
[Pau - Metz | H] odds=4.10, edge=0.255, p=0.306, stake=1.00
[Bastia - Grenoble | A] odds=5.20, edge=0.234, p=0.237, stake=1.00
Bankroll used: 8.84 €


In [66]:
import numpy as np
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (np.integer, np.int_)):
            return int(obj)
        elif isinstance(obj, (np.floating, np.float_)):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return super().default(obj)

In [67]:
bets_dict_list = [bet.to_dict() for bet in selected_bet]


with open("bets.json", "w") as f:
    json.dump(bets_dict_list, f, indent=4, cls=NumpyEncoder)

In [68]:
#pm.summary(model.trace)

In [69]:
from scipy.stats import skellam
def _skellam_post_probs(
    lh: np.ndarray, la: np.ndarray
) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    """Probabilités a posteriori (vecteurs) : home‑win, draw, away‑win pour des échantillons
    λ_home, λ_away de même longueur."""
    p_home = 1 - skellam.cdf(0, lh, la)  # P(diff > 0)
    p_draw = skellam.pmf(0, lh, la)  # P(diff = 0)
    p_away = skellam.cdf(-1, lh, la)  # P(diff < 0)
    return p_home, p_draw, p_away


In [84]:
def bayesian_kelly(
        list_bet: list[Bet],               # shape (J,)   : cotes décimales du bookmaker
        lambda_samples: dict[str, tuple[np.ndarray, np.ndarray]],          # shape (K, J) : échantillon posterior des probabilités
        bankroll: float=100,       # capital initial (1 = 100 %)
        summary: Literal["mean", "quantile"] ="quantile", # "mean" ou "quantile"
        alpha: float=0.3,          # si summary=="quantile"
        global_fraction: float=0.5,# λ (½‑Kelly global)
        per_bet_cap: float =0.90,   # plafond f_j max
    ):
    """
    Renvoie les fractions de bankroll à miser sur chacun des J matchs.
    Les négatives sont tronquées à 0 (on ignore les edges négatifs).
    """
    mapping_match = {"H":0, "D":1, "A": 2}
    bets_w_stake = []
    for bet in list_bet:
        # (b)  Kelly instantané pour chaque tirage
        lambda_h, lambda_a = lambda_samples[bet.match_id]
        probas = _skellam_post_probs(lh=lambda_h, la=lambda_a)
        p_samples = probas[mapping_match[bet.market]]
        X = bet.odds - 1.0                               # gain unitaire si victoire
        f_kelly = (p_samples * bet.odds - (1 - p_samples)) / X   # shape (K, J)

        f_kelly = np.clip(f_kelly, 0.0, None)        # on ne short pas

        # (c)  Résumé prudent
        if summary == "mean":
            f_base = f_kelly.mean(axis=0)
        elif summary == "quantile":
            f_base = np.quantile(f_kelly, alpha, axis=0)
        else:
            raise ValueError("summary must be 'mean' or 'quantile'")

        # (d)  Application du facteur global λ
        f = global_fraction * f_base

        # (e)  Contraintes pratiques
        f = np.minimum(f, per_bet_cap)               # plafond par match

        # Mise absolue si bankroll différent de 1
        stake = bankroll * f
        if stake >0.:
            tmp_bet = bet
            tmp_bet.stake = int(round(stake, 0))
            bets_w_stake.append(tmp_bet)

    sum_stake = 0
    possible_return = 0
    for bet in bets_w_stake:
        print(f"{bet}")
        sum_stake += bet.stake
        possible_return += bet.odds*bet.stake
    print(f"Bankroll used: {sum_stake:.2f} €")
    print(f"Possible return: {possible_return:.2f} €")
    return bets_w_stake


In [ ]:

def kelly_shrinkage(
    list_bet,
    lambda_samples,
    per_bet_cap=0.10,
    bankroll_cap=0.30,
    bankroll=100,
    lambda_global=0.25,          # <- ¼-Kelly by default
):
    mapping_match = {"H": 0, "D": 1, "A": 2}
    bets_w_stake = []

    for bet in list_bet:
        λh, λa = lambda_samples[bet.match_id]
        probs  = _skellam_post_probs(λh, λa)                    # shape (K,3) or (3,)
        p      = probs[mapping_match[bet.market]]

        # --- posterior mean & variance across samples -------------
        mu  = p.mean(0)                                          # scalar for that market
        var = p.var(0, ddof=1)

        # guard against var=0                                   ← prevents s=1 always
        if var == 0:
            var = 1e-9

        shrink = mu * (1-mu) / (mu*(1-mu) + var)                # s ∈ (0,1]

        # --- fractional-Kelly ------------------------------------
        b     = bet.odds - 1.0
        full  = (mu * bet.odds - (1 - mu)) / b                  # f⋆
        f     = lambda_global * shrink * full                   # λ-Kelly with shrink

        f     = np.clip(f, 0.0, per_bet_cap)                    # single-bet cap
        # bankroll-wide cap (if you really need it)
        # (collect f’s, rescale once outside the loop for clarity)

        stake = bankroll * f
        if stake > 0:
            bet.stake = int(round(stake))
            bets_w_stake.append(bet)

    # optional: rescale all stakes here if Σf > bankroll_cap
    total_fraction = sum(b.stake for b in bets_w_stake) / bankroll
    if total_fraction > bankroll_cap:
        scale = bankroll_cap / total_fraction
        for bet in bets_w_stake:
            bet.stake = int(round(bet.stake * scale))
    sum_stake = 0
    possible_return = 0
    for bet in bets_w_stake:
        print(f"{bet}")
        sum_stake += bet.stake
        possible_return += bet.odds*bet.stake
    print(f"Bankroll used: {sum_stake:.2f} €")
    print(f"Possible return: {possible_return:.2f} €")
    return bets_w_stake        # fractions et mises absolues

In [87]:
bayesian_kelly(list_bet=enhanced_list_bet, lambda_samples=list_samples, bankroll=20, summary="mean", global_fraction=1.)

[Guingamp - FC Martigues | A] odds=7.20, edge=0.415, p=0.196, stake=2.00
[Rodez - Paris FC | H] odds=2.95, edge=0.348, p=0.457, stake=8.00
[Red Star - Clermont | A] odds=3.65, edge=0.256, p=0.344, stake=5.00
[Pau - Metz | H] odds=4.10, edge=0.255, p=0.306, stake=4.00
[Bastia - Grenoble | A] odds=5.20, edge=0.234, p=0.237, stake=2.00
Bankroll used: 21.00 €
Possible return: 83.05 €


[Bet(match_id='Guingamp - FC Martigues', market='A', odds=7.2, edge_mean=0.4147971590267324, prob_mean=0.1964996054203795, edge_std=0.447112360145378, prob_edge_pos=0.816, stake=2),
 Bet(match_id='Rodez - Paris FC', market='H', odds=2.95, edge_mean=0.3475090933698626, prob_mean=0.45678274351520765, edge_std=0.2852163463582172, prob_edge_pos=0.8885, stake=8),
 Bet(match_id='Red Star - Clermont', market='A', odds=3.65, edge_mean=0.2561424828445391, prob_mean=0.34414862543686, edge_std=0.2721221382494443, prob_edge_pos=0.8226666666666667, stake=5),
 Bet(match_id='Pau - Metz', market='H', odds=4.1, edge_mean=0.25548584409357866, prob_mean=0.3062160595350192, edge_std=0.3475483505361055, prob_edge_pos=0.75875, stake=4),
 Bet(match_id='Bastia - Grenoble', market='A', odds=5.2, edge_mean=0.23434221014949275, prob_mean=0.23737350195182555, edge_std=0.32937590555444957, prob_edge_pos=0.7481666666666666, stake=2)]

In [88]:
kelly_shrinkage(list_bet=enhanced_list_bet, lambda_samples=list_samples, bankroll=20)

1.924910619298012
2.0
2.0
2.0
2.0
[Guingamp - FC Martigues | A] odds=7.20, edge=0.415, p=0.196, stake=2.00
[Rodez - Paris FC | H] odds=2.95, edge=0.348, p=0.457, stake=2.00
[Red Star - Clermont | A] odds=3.65, edge=0.256, p=0.344, stake=2.00
[Pau - Metz | H] odds=4.10, edge=0.255, p=0.306, stake=2.00
[Bastia - Grenoble | A] odds=5.20, edge=0.234, p=0.237, stake=2.00
Bankroll used: 10.00 €
Possible return: 46.20 €


[Bet(match_id='Guingamp - FC Martigues', market='A', odds=7.2, edge_mean=0.4147971590267324, prob_mean=0.1964996054203795, edge_std=0.447112360145378, prob_edge_pos=0.816, stake=2),
 Bet(match_id='Rodez - Paris FC', market='H', odds=2.95, edge_mean=0.3475090933698626, prob_mean=0.45678274351520765, edge_std=0.2852163463582172, prob_edge_pos=0.8885, stake=2),
 Bet(match_id='Red Star - Clermont', market='A', odds=3.65, edge_mean=0.2561424828445391, prob_mean=0.34414862543686, edge_std=0.2721221382494443, prob_edge_pos=0.8226666666666667, stake=2),
 Bet(match_id='Pau - Metz', market='H', odds=4.1, edge_mean=0.25548584409357866, prob_mean=0.3062160595350192, edge_std=0.3475483505361055, prob_edge_pos=0.75875, stake=2),
 Bet(match_id='Bastia - Grenoble', market='A', odds=5.2, edge_mean=0.23434221014949275, prob_mean=0.23737350195182555, edge_std=0.32937590555444957, prob_edge_pos=0.7481666666666666, stake=2)]